In [1]:
import torch
import torch.nn as nn
from datasets import IndexedDataset, load_data
from torch.utils.data import DataLoader

from utils import get_args
from architectures import load_architecture

from architectures import load_architecture, add_lora, set_lora_gradients  # load_statedict

from torch.utils.data import DataLoader
import numpy as np

import math
from functools import partial
from typing import Union, Tuple  # Import Union and Tuple for type annotations

import torch.nn.functional as F
from torch import optim

@torch.inference_mode()
def _get_masks(activations, tau: float, ineq_type: str) -> torch.Tensor:
    """
    Computes the mask for a given set of activations based on the inequality type.
    The returned mask has True where neurons satisfy the condition and False otherwise.
    """
    masks = []

    for name, activation in activations.items():
        # Calculate the mean activation per neuron
        if activation.ndim == 4:
            # Conv layer
            score = activation.abs().mean(dim=(0, 2, 3))
        else:
            # Linear layer
            score = activation.abs().mean(dim=0)

        # Normalize the score
        normalized_score = score / (score.mean() + 1e-9)
        layer_mask = torch.zeros_like(normalized_score, dtype=torch.bool)

        if ineq_type == 'leq':
            layer_mask[normalized_score <= tau] = True
        elif ineq_type == 'geq':
            layer_mask[normalized_score >= tau] = True
        elif ineq_type == 'eq':
            layer_mask[torch.isclose(normalized_score, torch.zeros_like(normalized_score))] = True
        else:
            raise ValueError(f"Invalid inequality type: {ineq_type}")

        masks.append(layer_mask)

    return masks

@torch.inference_mode()
def _get_activation(name: str, activations):
    """Fetches and stores the activations of a network layer."""

    def hook(layer: Union[nn.Linear, nn.Conv2d], input: Tuple[torch.Tensor, ...], output: torch.Tensor) -> None:
        """
        Get the activations of a layer with ReLU nonlinearity.
        ReLU has to be called explicitly here because the hook is attached to the conv/linear layer.
        """
        activations[name] = F.relu(output)

    return hook

@torch.inference_mode()
def run_redo(
    obs: torch.Tensor,
    model: nn.Module,
):
    """
    Checks the number of zero, dormant, and overactive neurons for a given model.

    Returns a dictionary containing the counts and fractions.
    """

    activations = {}
    activation_getter = partial(_get_activation, activations=activations)

    # Register hooks for all Conv2d and Linear layers to calculate activations
    handles = []
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            handles.append(module.register_forward_hook(activation_getter(name)))

    # Calculate activations
    _ = model(obs)

    # Compute zero masks (neurons with zero normalized activation)
    zero_masks = _get_masks(activations, 0.0, 'eq')
    zero_count = sum([torch.sum(mask).item() for mask in zero_masks])

    # Compute dormant masks (excluding zero neurons)
    dormant_masks = _get_masks(activations, 0.01, 'leq')
    # Exclude zero neurons from dormant neurons
    adjusted_dormant_masks = [dormant_mask & (~zero_mask) for dormant_mask, zero_mask in zip(dormant_masks, zero_masks)]
    dormant_count = sum([torch.sum(mask).item() for mask in adjusted_dormant_masks])

    # Compute overactive masks
    overactive_masks = _get_masks(activations, 3.0, 'geq')
    overactive_count = sum([torch.sum(mask).item() for mask in overactive_masks])

    # Compute total neurons
    total_neurons = sum([mask.numel() for mask in zero_masks])

    # Compute fractions
    zero_fraction = zero_count / total_neurons
    dormant_fraction = dormant_count / total_neurons
    overactive_fraction = overactive_count / total_neurons

    # Remove the hooks
    for handle in handles:
        handle.remove()

    return {
        "total_neurons": total_neurons,
        "zero_fraction": zero_fraction,
        "zero_count": zero_count,
        "dormant_fraction": dormant_fraction,
        "dormant_count": dormant_count,
        "overactive_fraction": overactive_fraction,
        "overactive_count": overactive_count,
    }

In [43]:
from omegaconf import OmegaConf
from datasets import load_data
import numpy as np
import torch
from torchvision import datasets, transforms

config = OmegaConf.load("./configs/default_config.yaml")

# Create white and black images using NumPy
# white_image = np.ones((224, 224, 3), dtype=np.uint8) * 255
# black_image = np.zeros((224, 224, 3), dtype=np.uint8)
# white_tensor = torch.from_numpy(white_image).permute(2, 0, 1).float() / 255.0
# black_tensor = torch.from_numpy(black_image).permute(2, 0, 1).float() / 255.0
# white_tensor = white_tensor.unsqueeze(0)  # Shape: (1, 3, 224, 224)
# black_tensor = black_tensor.unsqueeze(0)  # Shape: (1, 3, 224, 224)

config.dataset = 'Imagenette'
train_dataset, val_dataset, test_dataset, N, train_transform, transform = load_data(False, config) 
test_dataset = IndexedDataset(test_dataset, transform,  N,) 
testloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=3, pin_memory=True)


from collections import defaultdict

def average_dicts(dicts):
    accumulator = defaultdict(float)
    count = defaultdict(int)

    for d in dicts:
        for key, value in d.items():
            accumulator[key] += value
            count[key] += 1

    return {key: accumulator[key] / count[key] for key in accumulator}

def generate_random_observations(batch_size, image_size, channels, value_range=(0, 1), apply_transforms=None):

    # Generate random images within the specified range
    min_val, max_val = value_range
    random_images = torch.rand(batch_size, channels, image_size, image_size) * (max_val - min_val) + min_val

    # Apply transforms if provided
    if apply_transforms:
        # Convert each image to a PIL Image, apply transforms, and stack them back together
        random_images = torch.stack([
            apply_transforms(transforms.ToPILImage()(img)) for img in random_images
        ])

    return random_images


transform = transforms.Compose([transforms.Resize((224, 224)),  # Resize images to 224x224
                                    transforms.ToTensor(),
                                    transforms.Normalize( mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] ),])


backbones = [ 'convnext_base',  'convnext_base.fb_in22k', 'robust_convnext_base',
              'convnext_tiny',  'convnext_tiny.fb_in22k', 'robust_convnext_tiny', 
              'deit_small_patch16_224.fb_in1k', 'random_deit_small_patch16_224' ,
              'vit_base_patch16_224.augreg_in1k', 'vit_base_patch16_224.augreg_in21k', 'robust_vit_base_patch16_224'   ]

#   'robust_wideresnet_28_10', 'wideresnet_28_10',

backbones_result = {}
for backbone_name in backbones:
    # print(backbone_name)
    print( backbone_name )

    backbone_result = {
        "total_neurons":[],
        "zero_fraction": [],
        "zero_count": [],
        "dormant_fraction": [],
        "dormant_count": [],
        "overactive_fraction": [],
        "overactive_count": [],
    }

    config.backbone = backbone_name
    model = load_architecture(False, config, N = 10, )

    # result = run_redo(white_tensor, model) # compute amount of zero, dormant and overactive neurons
    # print('white', result)
    # result = run_redo(black_tensor, model) # compute amount of zero, dormant and overactive neurons
    # print('black', result)

    result_random = []
    for _ in range(1):

        data = generate_random_observations(1, 224, 3, value_range=(0, 255), apply_transforms=transform)

        result_random.append(  run_redo(data, model) )# compute amount of zero, dormant and overactive neurons


        if batch_id == 100:
            break

    result_random = average_dicts(result_random)
    print('random', result_random)
    
    result_image = []
    for batch_id, batch in  enumerate(testloader) :

        data, target, idxs = batch

        result_image.append( run_redo(data, model) ) # compute amount of zero, dormant and overactive neurons

        if batch_id == 100:
            break

    result_image = average_dicts(result_image)
    print('image', result_image)

    print()

datadir /home/mheuillet/Desktop/robust_training/data
convnext_base
random {'total_neurons': 21280.0, 'zero_fraction': 0.21527255639097745, 'zero_count': 4581.0, 'dormant_fraction': 0.05996240601503759, 'dormant_count': 1276.0, 'overactive_fraction': 0.07664473684210527, 'overactive_count': 1631.0}
image {'total_neurons': 21280.0, 'zero_fraction': 0.035473274771086145, 'zero_count': 754.8712871287129, 'dormant_fraction': 0.017965085982282438, 'dormant_count': 382.2970297029703, 'overactive_fraction': 0.0501549356063426, 'overactive_count': 1067.2970297029703}

convnext_base.fb_in22k
random {'total_neurons': 21280.0, 'zero_fraction': 0.22410714285714287, 'zero_count': 4769.0, 'dormant_fraction': 0.06024436090225564, 'dormant_count': 1282.0, 'overactive_fraction': 0.0787124060150376, 'overactive_count': 1675.0}
image {'total_neurons': 21280.0, 'zero_fraction': 0.02866634407801682, 'zero_count': 610.019801980198, 'dormant_fraction': 0.016301738256532423, 'dormant_count': 346.9009900990099,

In [3]:

from torch.nn.parallel import DistributedDataParallel as DDP

from utils import ActivationTracker, register_hooks, compute_stats
from architectures import load_architecture, CustomModel
import numpy as np
import torch
from omegaconf import OmegaConf
from datasets import load_data
import torch.distributed as dist
import numpy as np
import torch

config = OmegaConf.load("./configs/default_config.yaml")


N = 10
B = 2  

# Create a white image using NumPy
white_image = np.ones((224, 224, 3), dtype=np.uint8) * 255

# Convert to PyTorch tensor
white_tensor = torch.from_numpy(white_image).permute(2, 0, 1).float() / 255.0  # Shape: (3, 224, 224)

# Add batch dimension and repeat for batch size B
white_tensor = white_tensor.unsqueeze(0).repeat(B, 1, 1, 1)  # Shape: (B, 3, 224, 224)

# Move the tensor to GPU
white_tensor = white_tensor.to('cuda')

print(white_tensor.shape)  # Verify shape: (B, 3, 224, 224)

config.backbone = 'convnext_tiny'
model = load_architecture(False, config, N, )
model = CustomModel(config, model, )
model.to('cuda')

model.eval()

tracker_nat = ActivationTracker()
tracker_adv = ActivationTracker()

handles = register_hooks(model, tracker_nat, tracker_adv)

model.current_task = 'infer'
model(white_tensor)

# # Compute neuron statistics
res_nat = compute_stats(tracker_nat.activations)
# res_adv = compute_stats(tracker_adv.activations)

print(res_nat)
# print(res_adv)

torch.Size([2, 3, 224, 224])
{'total_neurons': 8872, 'zero_count': 1561, 'dormant_count': 401, 'overactive_count': 624}


In [2]:

from torch.nn.parallel import DistributedDataParallel as DDP

from utils import ActivationTracker, register_hooks, compute_stats
from architectures import load_architecture, CustomModel
import numpy as np
import torch
from omegaconf import OmegaConf
from datasets import load_data
import torch.distributed as dist
import numpy as np
import torch

config = OmegaConf.load("./configs/default_config.yaml")


N = 10
B = 2

# Create a white image using NumPy
white_image = np.ones((224, 224, 3), dtype=np.uint8) * 255

# Convert to PyTorch tensor
white_tensor = torch.from_numpy(white_image).permute(2, 0, 1).float() / 255.0  # Shape: (3, 224, 224)

# Add batch dimension and repeat for batch size B
white_tensor = white_tensor.unsqueeze(0).repeat(B, 1, 1, 1)  # Shape: (B, 3, 224, 224)

# Move the tensor to GPU
white_tensor = white_tensor.to('cuda')

print(white_tensor.shape)  # Verify shape: (B, 3, 224, 224)

config.backbone = 'convnext_tiny'
model = load_architecture(False, config, N, )
model = CustomModel(config, model, )
model.to('cuda')

model.eval()

tracker_nat = ActivationTracker()
tracker_adv = ActivationTracker()

handles = register_hooks(model, tracker_nat, tracker_adv)

model.current_task = 'infer'
model(white_tensor)

# # Compute neuron statistics
res_nat = compute_stats(tracker_nat.activations)
# res_adv = compute_stats(tracker_adv.activations)

print(res_nat)
print(tracker_nat.activations['base_model.stem.0'].shape)
# print(res_adv)

torch.Size([2, 3, 224, 224])
score tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 7.9517e-01, 0.0000e+00, 4.1610e-01, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 8.0445e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 3.3357e+00, 9.4621e-02, 0.0000e+00, 0.0000e+00, 2.5304e-01,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 6.3615e-03, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.4487e+00, 2.3436e-02, 1.3762e-01, 2.2767e-01, 2.7563e+00,
        0.0000e+00, 0.0000e+00, 4.3441e-03, 2.6298e-02, 0.0000e+00, 1.1334e-02,
        0.0000e+00, 5.1408e-02, 1.3963e-01, 8.8616e-03, 0.0000e+00, 0.0000e+00,
        3.9974e-02, 0.0000e+00, 1.5570e+00, 0.0000e+00, 3.9853e-02, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 6.7071e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        6.4512e-02, 3.2202e+00, 5.1580e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
     

In [ ]:
import torch
import torch.nn as nn
from functools import partial
from typing import Union, Tuple
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

class ActivationTrackerAggregated:
    def __init__(self):
        # For each layer: store cumulative sums and counts of activations
        self.sums = {}   # Will hold accumulated sums of absolute activations
        self.counts = {} # Will hold total counts of elements processed per neuron
        self.is_conv = {} # Keep track if layer is Conv (for dimension handling)

    def accumulate(self, name, activation):
        # Move to CPU for safety
        activation = activation.detach().cpu()

        # Check if it's a conv layer or linear layer by dimensions
        # Conv: (B, C, H, W)
        # Linear: (B, C)
        if activation.ndim == 4:
            # Conv layer
            # sum over batch, height, and width: result is shape (C,)
            abs_sum = activation.abs().sum(dim=(0, 2, 3))
            # count how many elements per channel
            b, c, h, w = activation.shape
            elem_count = b * h * w
            if name not in self.sums:
                self.sums[name] = torch.zeros(c, dtype=torch.float32)
                self.counts[name] = 0
                self.is_conv[name] = True
            self.sums[name] += abs_sum
            self.counts[name] += elem_count

        else:
            # Linear layer: (B, C)
            abs_sum = activation.abs().sum(dim=0)  # shape (C,)
            b, c = activation.shape
            elem_count = b
            if name not in self.sums:
                self.sums[name] = torch.zeros(c, dtype=torch.float32)
                self.counts[name] = 0
                self.is_conv[name] = False
            self.sums[name] += abs_sum
            self.counts[name] += elem_count

    def get_activations_mean(self):
        # Compute mean absolute activation per neuron
        activations_mean = {}
        for name in self.sums:
            activations_mean[name] = self.sums[name] / (self.counts[name] + 1e-9)
        return activations_mean

    def clear(self):
        self.sums = {}
        self.counts = {}
        self.is_conv = {}

def register_hooks_aggregated(model, tracker_nat, tracker_adv):
    handles = []
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            module._name = name

            def get_activation(mod, model):
                def hook(mod, input, output):
                    name = mod._name
                    if model.current_tracker == 'nat' and model.current_task == 'infer':
                        tracker_nat.accumulate(name, F.relu(output))
                    if model.current_tracker == 'adv' and model.current_task == 'infer':
                        tracker_adv.accumulate(name, F.relu(output))
                return hook

            handle = module.register_forward_hook(get_activation(module, model))
            handles.append(handle)
    return handles

@torch.inference_mode()
def _compute_masks_from_stats(activations_mean, is_conv, tau, ineq_type):
    masks = []
    for name, mean_vals in activations_mean.items():
        # mean_vals is per-channel (conv) or per-neuron (linear)
        score = mean_vals  # already mean per neuron over dataset
        # Normalize by its own mean
        normalized_score = score / (score.mean() + 1e-9)
        layer_mask = torch.zeros_like(normalized_score, dtype=torch.bool)

        if ineq_type == 'leq':
            layer_mask[normalized_score <= tau] = True
        elif ineq_type == 'geq':
            layer_mask[normalized_score >= tau] = True
        elif ineq_type == 'eq':
            layer_mask[torch.isclose(normalized_score, torch.zeros_like(normalized_score))] = True
        else:
            raise ValueError(f"Invalid inequality type: {ineq_type}")
        masks.append(layer_mask)
    return masks

@torch.inference_mode()
def compute_stats_aggregated(tracker):
    # Compute the mean activation values
    activations_mean = tracker.get_activations_mean()

    # Compute zero masks (neurons with zero normalized activation)
    zero_masks = _compute_masks_from_stats(activations_mean, tracker.is_conv, 0.0, 'eq')
    zero_count = sum([mask.sum().item() for mask in zero_masks])

    # Compute dormant masks (excluding zero neurons)
    dormant_masks = _compute_masks_from_stats(activations_mean, tracker.is_conv, 0.01, 'leq')
    adjusted_dormant_masks = [dormant & (~zero) for dormant, zero in zip(dormant_masks, zero_masks)]
    dormant_count = sum([mask.sum().item() for mask in adjusted_dormant_masks])

    # Compute overactive masks
    overactive_masks = _compute_masks_from_stats(activations_mean, tracker.is_conv, 3.0, 'geq')
    overactive_count = sum([mask.sum().item() for mask in overactive_masks])

    # Compute total neurons
    total_neurons = sum([mask.numel() for mask in zero_masks])

    return {
        "total_neurons": total_neurons,
        "zero_fraction": zero_count/total_neurons,
        "dormant_fraction": dormant_count/total_neurons,
        "overactive_fraction": overactive_count/total_neurons,
    }



from torch.nn.parallel import DistributedDataParallel as DDP

# from utils import ActivationTracker, register_hooks, compute_stats
from architectures import load_architecture, CustomModel
import numpy as np
import torch
from omegaconf import OmegaConf
from datasets import load_data
import torch.distributed as dist
import numpy as np
import torch

config = OmegaConf.load("./configs/default_config.yaml")

N = 10
B = 2

# Create a white image using NumPy
white_image = np.ones((224, 224, 3), dtype=np.uint8) * 255

# Convert to PyTorch tensor
white_tensor = torch.from_numpy(white_image).permute(2, 0, 1).float() / 255.0  # Shape: (3, 224, 224)

# Add batch dimension and repeat for batch size B
white_tensor = white_tensor.unsqueeze(0).repeat(B, 1, 1, 1)  # Shape: (B, 3, 224, 224)

# Move the tensor to GPU
white_tensor = white_tensor.to('cuda')

print(white_tensor.shape)  # Verify shape: (B, 3, 224, 224)

config.backbone = 'convnext_tiny'
model = load_architecture(False, config, N, )
model = CustomModel(config, model, )
model.to('cuda')

model.eval()

tracker_nat = ActivationTrackerAggregated()
tracker_adv = ActivationTrackerAggregated()

handles = register_hooks_aggregated(model, tracker_nat, tracker_adv)

model.current_task = 'infer'
model(white_tensor)

# # Compute neuron statistics
# res_nat = compute_stats2(tracker_nat)
# res_adv = compute_stats(tracker_adv.activations)

print( tracker_nat.sums['base_model.stem.0'].shape , tracker_nat.counts )
# print(res_adv)

model.current_task = 'infer'
model(white_tensor)

print( tracker_nat.sums['base_model.stem.0'].shape , tracker_nat.counts )




# # Compute neuron statistics
res_nat = compute_stats_aggregated(tracker_nat)
print(res_nat)

torch.Size([2, 3, 224, 224])
torch.Size([96]) {'base_model.stem.0': 6272, 'base_model.stages.0.blocks.0.conv_dw': 6272, 'base_model.stages.0.blocks.0.mlp.fc1': 43008, 'base_model.stages.0.blocks.0.mlp.fc2': 10752, 'base_model.stages.0.blocks.1.conv_dw': 6272, 'base_model.stages.0.blocks.1.mlp.fc1': 43008, 'base_model.stages.0.blocks.1.mlp.fc2': 10752, 'base_model.stages.0.blocks.2.conv_dw': 6272, 'base_model.stages.0.blocks.2.mlp.fc1': 43008, 'base_model.stages.0.blocks.2.mlp.fc2': 10752, 'base_model.stages.1.downsample.1': 1568, 'base_model.stages.1.blocks.0.conv_dw': 1568, 'base_model.stages.1.blocks.0.mlp.fc1': 43008, 'base_model.stages.1.blocks.0.mlp.fc2': 10752, 'base_model.stages.1.blocks.1.conv_dw': 1568, 'base_model.stages.1.blocks.1.mlp.fc1': 43008, 'base_model.stages.1.blocks.1.mlp.fc2': 10752, 'base_model.stages.1.blocks.2.conv_dw': 1568, 'base_model.stages.1.blocks.2.mlp.fc1': 43008, 'base_model.stages.1.blocks.2.mlp.fc2': 10752, 'base_model.stages.2.downsample.1': 392, 'ba

In [4]:
import pandas as pd

df = pd.DataFrame.from_dict(backbones_result).T

df.to_csv( 'neurons_results_{}.csv'.format(args.dataset) )

In [ ]:
import pandas as pd

dataset = 'EuroSAT'

df = pd.read_csv('neurons_results_{}.csv'.format(dataset), index_col=0)

# Replace underscores with spaces in column names and data
df.columns = df.columns.str.replace('_', ' ')
df = df.replace('_', ' ', regex=True)
df.index = df.index.str.replace('_', ' ')

# Rounding a DataFrame to 3 decimal places
df = df.round(3)

latex_code = df.to_latex(
    index=True,
    formatters={"name": str.upper},
    float_format="{}".format
)

# Print the LaTeX code
print(latex_code)


\begin{tabular}{lrrrrrrrrrrrrrr}
\toprule
 & mean total neurons & mean zero fraction & mean zero count & mean dormant fraction & mean dormant count & mean overactive fraction & mean overactive count & std total neurons & std zero fraction & std zero count & std dormant fraction & std dormant count & std overactive fraction & std overactive count \\
\midrule
convnext base & 21280.0 & 0.047 & 1004.16 & 0.163 & 3469.93 & 0.056 & 1200.18 & 0.0 & 0.017 & 359.605 & 0.035 & 737.983 & 0.005 & 112.553 \\
convnext base.fb in22k & 21280.0 & 0.043 & 921.53 & 0.158 & 3354.2 & 0.056 & 1183.79 & 0.0 & 0.017 & 355.33 & 0.035 & 742.071 & 0.005 & 112.939 \\
robust convnext base & 21280.0 & 0.103 & 2194.19 & 0.257 & 5472.24 & 0.069 & 1463.75 & 0.0 & 0.053 & 1127.165 & 0.077 & 1641.495 & 0.012 & 262.539 \\
convnext tiny & 8872.0 & 0.041 & 360.27 & 0.126 & 1115.94 & 0.049 & 432.75 & 0.0 & 0.017 & 154.956 & 0.037 & 325.717 & 0.007 & 57.75 \\
convnext tiny.fb in22k & 8872.0 & 0.062 & 551.18 & 0.175 & 1550.68

In [5]:
df

,mean_total_neurons,mean_zero_fraction,mean_zero_count,mean_dormant_fraction,mean_dormant_count,mean_overactive_fraction,mean_overactive_count,std_total_neurons,std_zero_fraction,std_zero_count,std_dormant_fraction,std_dormant_count,std_overactive_fraction,std_overactive_count
convnext_base,21280.0,0.047139,1003.12,0.163013,3468.92,0.056383,1199.83,0.0,0.016902,359.664602,0.034687,738.129524,0.005287,112.496760
convnext_base.fb_in22k,21280.0,0.043351,922.50,0.157668,3355.18,0.055629,1183.79,0.0,0.016660,354.518956,0.034830,741.185609,0.005301,112.813678
robust_convnext_base,21280.0,0.103116,2194.30,0.257160,5472.37,0.068804,1464.14,0.0,0.052942,1126.612387,0.077115,1641.006323,0.012328,262.334444
convnext_tiny,8872.0,0.040629,360.46,0.125803,1116.12,0.048785,432.82,0.0,0.017429,154.629584,0.036688,325.500024,0.006506,57.721292
convnext_tiny.fb_in22k,8872.0,0.062261,552.38,0.174923,1551.92,0.057359,508.89,0.0,0.020163,178.889227,0.039880,353.814434,0.005824,51.673571
robust_convnext_tiny,8872.0,0.183944,1631.95,0.318894,2829.23,0.054776,485.97,0.0,0.062596,555.355244,0.067971,603.036000,0.003891,34.522588
robust_wideresnet_28_10,10106.0,0.002412,24.38,0.029793,301.09,0.015400,155.63,0.0,0.000455,4.599522,0.002347,23.719652,0.001168,11.808179
wideresnet_28_10,10106.0,0.002266,22.90,0.026761,270.45,0.008736,88.29,0.0,0.000831,8.401786,0.001652,16.699925,0.000352,3.556107
deit_small_patch16_224.fb_in1k,8170378.0,0.688957,5629035.05,0.699874,5718238.12,0.088022,719174.09,0.0,0.002044,16697.380857,0.002359,19276.447727,0.001521,12425.288639
robust_deit_small_patch16_224,8170378.0,0.692757,5660087.78,0.705982,5768142.50,0.081712,667616.00,0.0,0.005185,42366.565605,0.005420,44279.603482,0.003662,29923.361248


In [2]:
from omegaconf import OmegaConf
from datasets import load_data
import numpy as np
import torch
from torchvision import datasets, transforms

config = OmegaConf.load("./configs/default_config.yaml")
N = 10

# Create white and black images using NumPy
white_image = np.ones((224, 224, 3), dtype=np.uint8) * 255
black_image = np.zeros((224, 224, 3), dtype=np.uint8)
white_tensor = torch.from_numpy(white_image).permute(2, 0, 1).float() / 255.0
black_tensor = torch.from_numpy(black_image).permute(2, 0, 1).float() / 255.0
white_tensor = white_tensor.unsqueeze(0)  # Shape: (1, 3, 224, 224)
black_tensor = black_tensor.unsqueeze(0)  # Shape: (1, 3, 224, 224)


backbones = [ 
            #   'convnext_base',  'convnext_base.fb_in22k', 'robust_convnext_base',
              'convnext_tiny' #'robust_convnext_tiny' 'random_convnext_tiny'  'convnext_tiny.fb_in22k', 

            #   'robust_wideresnet_28_10', 'wideresnet_28_10',
            #   'deit_small_patch16_224.fb_in1k',
            #   'robust_deit_small_patch16_224',
            #   'vit_base_patch16_224.augreg_in1k',
            #   'vit_base_patch16_224.augreg_in21k',
            #   'robust_vit_base_patch16_224'
                ]

backbones_result = {}
for backbone_name in backbones:
    # print(backbone_name)

    backbone_result = {
        "total_neurons":[],
        "zero_fraction": [],
        "zero_count": [],
        "dormant_fraction": [],
        "dormant_count": [],
        "overactive_fraction": [],
        "overactive_count": [],
    }

    config.backbone = backbone_name
    model = load_architecture(False, config, N, )

    from utils import ActivationTracker, register_hooks, compute_stats

    tracker_nat = ActivationTracker()
    tracker_adv = ActivationTracker()

    model.current_tracker = 'nat'
    handles = register_hooks(model, tracker_nat, tracker_adv)

    stats = {'nb_correct_nat': 0, 'nb_correct_adv': 0, 'nb_examples': 0}
    stats_nat = { "zero_count": 0, "dormant_count": 0, "overactive_count": 0, "total_neurons": 0 }
    stats_adv = { "zero_count": 0, "dormant_count": 0, "overactive_count": 0, "total_neurons": 0 }
            
    # print('Evaluate the model on natural data', rank, flush=True)
    model.current_tracker = 'nat'  # Set the tracker to natural data
    nat_outputs = model(white_tensor)
    _, preds_nat = torch.max(nat_outputs.data, 1)

    # print('Evaluate the model on adversarial examples', rank, flush=True)
    model.current_tracker = 'adv'  # Set the tracker to adversarial data
    adv_outputs = model(black_tensor)
    _, preds_adv = torch.max(adv_outputs.data, 1)

    stats_nat = compute_stats(tracker_nat.activations)
    stats_adv = compute_stats(tracker_adv.activations)

    print('white', stats_nat)
    print('black', stats_adv)

    # Clear activations for next batch
    tracker_nat.activations.clear()
    tracker_adv.activations.clear()

    result = run_redo(white_tensor, model) # compute amount of zero, dormant and overactive neurons

    print('white', result)

    result = run_redo(black_tensor, model) # compute amount of zero, dormant and overactive neurons

    print('black', result)

    print()

white {'total_neurons': 8872, 'zero_count': 1560, 'dormant_count': 401, 'overactive_count': 623}
black {'total_neurons': 8872, 'zero_count': 1838, 'dormant_count': 407, 'overactive_count': 617}
white {'total_neurons': 8872, 'zero_fraction': 0.17583408476104598, 'zero_count': 1560, 'dormant_fraction': 0.04519837691614067, 'dormant_count': 401, 'overactive_fraction': 0.07022091974752029, 'overactive_count': 623}
black {'total_neurons': 8872, 'zero_fraction': 0.20716862037871955, 'zero_count': 1838, 'dormant_fraction': 0.04587466185752931, 'dormant_count': 407, 'overactive_fraction': 0.06954463480613166, 'overactive_count': 617}

